In [1]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v1.1)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.8
Method: manual login


In [3]:
search_Var = "[associated_grant_ids+associated_publication+associated_publication_id+authors+category_bra+category_for+category_for_2020+category_hra+category_hrcs_hc+category_hrcs_rac+category_icrp_cso+category_icrp_ct+category_rcdc+category_sdg+date+date_created+date_embargo+date_inserted+date_modified+description+dimensions_url+doi+figshare_url+funder_countries+funders+id+journal+keywords+language_desc+language_title+license_name+license_url+repository+repository+research_org_cities+research_org_countries+research_org_states+research_orgs+researchers+score+title+year]"


In [33]:
def researcherIds(name):
    res = dsl.query_iterative(f"""search researchers for "{name}" return researchers""")
    listDict = res.json['researchers']
    allIds = set()
    for idv in listDict:
        allIds.add(idv['id'])

    allIds = list(allIds)  # Convert set back to a list if needed
    print(f'Total of {len(allIds)} unique ids for {name}: ')
    print(allIds)

    if len(allIds) > 500:
        batched_ids = [allIds[i:i+500] for i in range(0, len(allIds), 500)]
        allWorkingIDs = set()
        
        for batch in batched_ids:
            query_ids = ', '.join([f'"{id_val}"' for id_val in batch])
            query = f'search researchers where id in [{query_ids}] return researchers[id+obsolete+redirect]'
            
            res2 = dsl.query_iterative(query)
            listDict2 = res2.json['researchers']
            
            for person in listDict2:
                if person['obsolete'] == 0:  # Current Working Id(s)
                    allWorkingIDs.add(person["id"])
                else:
                    for ids in person['redirect']:
                        allWorkingIDs.add(ids)
        
        print(f'++++++++ FINAL {len(allWorkingIDs)} Working ID(s) +++++++++++')  
        print(allWorkingIDs)
        
    else:
        query_ids = ', '.join([f'"{id_val}"' for id_val in allIds])
        query = f'search researchers where id in [{query_ids}] return researchers[id+obsolete+redirect]'
        
        res2 = dsl.query_iterative(query)
        listDict2 = res2.json['researchers']
        allWorkingIDs = set()
        
        for person in listDict2:
            if person['obsolete'] == 0:  # Current Working Id(s)
                allWorkingIDs.add(person["id"])
            else:
                for ids in person['redirect']:
                    allWorkingIDs.add(ids)
        
        print(f'++++++++ FINAL {len(allWorkingIDs)} Working ID(s) +++++++++++')  
        print(allWorkingIDs)

    
    return allWorkingIDs
#testIDs = researcherIds("MICHAEL AGUS")
#Gareth L Nealon
testIDs = researcherIds("Gareth Nealon")

Starting iteration with limit=1000 skip=0 ...
0-3 / 3 (0.42s)
===
Records extracted: 3
Starting iteration with limit=1000 skip=0 ...


Total of 3 unique ids for Gareth Nealon: 
['ur.01146047701.41', 'ur.016071434137.11', 'ur.015007635567.03']


0-3 / 3 (0.37s)
===
Records extracted: 3


++++++++ FINAL 1 Working ID(s) +++++++++++
{'ur.01146047701.41'}


In [27]:
#using researcher name to find datasets
def findingDatasets(name):
    all_datasets = {}
    res = dsl.query_iterative(f"""search datasets where authors = "{name}" return datasets{search_Var}""")
    number = res.json['_stats']['total_count']
    typeDict = {}
    print(f'{name} has {number} datasets')
    
    listDict = res.json['datasets']
    all_datasets[name] = listDict
        
    return all_datasets
        
findingDatasets("Simone Vigod")

Starting iteration with limit=1000 skip=0 ...
0-8 / 8 (0.48s)
===
Records extracted: 8


Simone Vigod has 8 datasets


{'Simone Vigod': [{'associated_publication': {'doi': '10.1371/journal.pone.0284389',
    'id': 'pub.1157499910',
    'pmcid': 'PMC10132527',
    'pmid': '37099524'},
   'associated_publication_id': 'pub.1157499910',
   'authors': [{'name': 'Orges Alabaku'},
    {'name': 'Alyssa Yang'},
    {'name': 'Shenthuraan Tharmarajah'},
    {'name': 'Katie Suda'},
    {'name': 'Simone Vigod'},
    {'name': 'Mina Tadrous'}],
   'category_for': [{'id': '80023', 'name': '52 Psychology'},
    {'id': '80003', 'name': '32 Biomedical and Clinical Sciences'},
    {'id': '80232', 'name': '5202 Biological Psychology'},
    {'id': '80057', 'name': '3214 Pharmacology and Pharmaceutical Sciences'},
    {'id': '80003', 'name': '32 Biomedical and Clinical Sciences'},
    {'id': '80023', 'name': '52 Psychology'}],
   'category_for_2020': [{'id': '80023', 'name': '52 Psychology'},
    {'id': '80003', 'name': '32 Biomedical and Clinical Sciences'},
    {'id': '80232', 'name': '5202 Biological Psychology'},
    {'i